In [1]:
library(dplyr)
suppressPackageStartupMessages(library(ggforce))
suppressPackageStartupMessages(library(scatterpie))
suppressPackageStartupMessages(library(qgraph))
library(readxl)
source('fun.R')
setwd('~/workspace/Mida_collab/')


载入程序包：‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



载入程序包：‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union




In [2]:
nodes_meta <- read.csv('CellOracle/output/nodes_meta.csv') 
links <- read.csv('CellOracle/output/links_filtered.csv') %>% filter(coef_abs >=0.02)
all_genes_exp <- read.csv('CellOracle/all_genes_exp.csv') %>% filter(expression >= 0.04) %>% filter(pct >= 0.05)
all_TFs <- read.csv('TF_perturbation/all_TFs_perturbation.csv',row.names = 1) %>% .$all_TFs
GSEA_res <- read.csv('CellOracle/output/GSEA_res_df_exp0.04pct0.05size20.csv') %>% filter(padj.2<=0.05)
module_gene <- read.csv('CellOracle/gradient_table2.vRG.tsv',sep='\t') %>% filter(flag=='Pass')

In [3]:
nodes_meta_subset <- nodes_meta %>% filter(flag=='Pass')

In [4]:
unique(GSEA_res$module)

[1] "non-pseudoage" "3"             "2"             "5"            
[5] "4"

In [5]:
module4_genes <- read_excel('CellOracle/RGG_pickgene_for_network.xlsx',sheet=1)[[1]]
module2_genes <- read_excel('CellOracle/RGG_pickgene_for_network.xlsx',sheet=2)[[1]]
module3_genes <- module_gene %>% filter(Module == '3') %>% .$gene
module5_genes <- module_gene %>% filter(Module == '5') %>% .$gene

In [6]:
all_targets <- c(module2_genes,module4_genes,module3_genes,module5_genes) %>% unique()

In [7]:
all_TFs <- all_TFs[!(all_TFs %in% c('FOXN2','SREBF2'))]

In [8]:
links <- links %>% filter(source %in% all_TFs) %>% filter(target %in% all_targets) %>% filter(source %in% all_genes_exp$X) %>% filter(target %in% all_genes_exp$X)
lapply(unique(links$source),function(TF){
    links_tmp <- links %>% filter(source==TF)
    regulon <- c(TF,links_tmp$coef_mean) %>%
                    setNames(c(TF,links_tmp$target))
    return(regulon)
}) -> regulons
names(regulons) <- unique(links$source)
regulons <- list(vRG = regulons)

In [9]:
net_pre <- PrepareNet(regulon = regulons)

Warning message in data.frame(from = xx[1], to = names(regu), weight = regu, stringsAsFactors = FALSE):
“短变量里找到了行名，因此不用”
Warning message in data.frame(from = xx[1], to = names(regu), weight = regu, stringsAsFactors = FALSE):
“短变量里找到了行名，因此不用”
Warning message in data.frame(from = xx[1], to = names(regu), weight = regu, stringsAsFactors = FALSE):
“短变量里找到了行名，因此不用”
Warning message in data.frame(from = xx[1], to = names(regu), weight = regu, stringsAsFactors = FALSE):
“短变量里找到了行名，因此不用”
Warning message in data.frame(from = xx[1], to = names(regu), weight = regu, stringsAsFactors = FALSE):
“短变量里找到了行名，因此不用”
Warning message in data.frame(from = xx[1], to = names(regu), weight = regu, stringsAsFactors = FALSE):
“短变量里找到了行名，因此不用”
Warning message in data.frame(from = xx[1], to = names(regu), weight = regu, stringsAsFactors = FALSE):
“短变量里找到了行名，因此不用”
Warning message in data.frame(from = xx[1], to = names(regu), weight = regu, stringsAsFactors = FALSE):
“短变量里找到了行名，因此不用”
Warning message in data.frame(fr

In [10]:
all_genes_in_plot <- net_pre$meta$gene %>% unique()

In [11]:
net_pre$markers <- NULL

In [12]:
TF_module2 <- net_pre$links %>% filter(to %in% module2_genes) %>% .$from %>% unique()
net_pre$markers$Module2 <- unique(c(module2_genes,TF_module2))
TF_module4 <- net_pre$links %>% filter(to %in% module4_genes) %>% .$from %>% unique()
net_pre$markers$Module4 <- unique(c(module4_genes,TF_module4))
TF_module3 <- net_pre$links %>% filter(to %in% module3_genes) %>% .$from %>% unique()
net_pre$markers$Module3 <- unique(c(module3_genes,TF_module3))
TF_module5 <- net_pre$links %>% filter(to %in% module5_genes) %>% .$from %>% unique()
net_pre$markers$Module5 <- unique(c(module5_genes,TF_module5))

In [13]:
module_gene <- module_gene %>% filter(flag == 'Pass') %>% filter(Module %in% c('4','5','3','2'))
lapply(c('4','5','3','2'),function(module){
    tmp <- module_gene %>% filter(Module==module) %>% .$gene
    return(tmp)
}) -> gene_module
names(gene_module) <- c('4','5','3','2')

In [14]:
GSEA_bi <- read.csv('CellOracle/output/GSEA_mat_bi_exp0.04pct0.05size20.csv',row.names = 1)
colnames(GSEA_bi) <- c('4','5','3','2')

In [15]:
raw_links <- read.csv('CellOracle/output/links_filtered.csv')
lapply(c('4','5','3','2'),function(used_module){
    tmp_res <- GSEA_res %>% filter(module == used_module) %>% arrange(desc(NES)) %>% select(c('tf','module','NES','padj.2'))
    tmp_res <- tmp_res %>% filter(tf %in% all_TFs)
    return(tmp_res)
    }) %>% do.call(rbind,.) -> all_TFs_df

i <- 1 
lapply(all_TFs_df$tf,function(TF){
    tmp_links <- raw_links %>% filter(source==TF) %>% filter(target %in% module_gene$gene)
    sapply(names(gene_module),function(module){
        if (!is.na(GSEA_bi[TF,module]) && GSEA_bi[TF,module]){
            return(sum(tmp_links$target %in% gene_module[[module]])/nrow(tmp_links)/length(gene_module[[module]]))}
        else {
            return(0)
        }
    }) -> tmp_proportion
    tmp_proportion <- tmp_proportion/sum(tmp_proportion)
    all_TFs_df[i,names(gene_module)] <<- tmp_proportion
    i <<- i+1
    # return(NULL)
})

[[1]]
[1] 2

[[2]]
[1] 3

[[3]]
[1] 4

[[4]]
[1] 5

[[5]]
[1] 6

[[6]]
[1] 7

[[7]]
[1] 8

[[8]]
[1] 9

[[9]]
[1] 10

[[10]]
[1] 11

[[11]]
[1] 12

[[12]]
[1] 13

[[13]]
[1] 14

In [16]:
region_colors <- c(Sulcus='#1e807a',  Adjacent = '#f5d731',Distant = '#d92523')
module_colors <- c(Module4='#7fb2d3',Module5='#8dd3c9',Module3='#ffb55f',Module2='#fc7f71')

In [17]:
all_TFs_df <- all_TFs_df[!duplicated(all_TFs_df$tf),]

In [18]:
rownames(all_TFs_df) <- all_TFs_df$tf
lapply(all_TFs_df$tf,function(TF){
   tmp_df <- all_TFs_df[,c('2','4','3','5')]
    res <- unname(unlist(as.vector(tmp_df[TF,])))
    res <- c(res,0)
    return(res)
}) %>% setNames(all_TFs_df$tf) -> pie_dict

In [19]:
region_markers <- list()
region_markers$Distant <- module_gene %>% filter(cluster == 'Distant') %>% .$gene
region_markers$Sulcus <- module_gene %>% filter(cluster == 'Sulcus') %>% .$gene

In [20]:
plot_tfnet_signal_final(netpre_object = net_pre,
                        vertice_anno_gset = net_pre$markers,
                        vertice_anno_shape = region_markers,
                        vertice_size = c(4, 0.5, 6),
                        label.scale = c(1, 1.5),
                        edge_width = c(0.2, 10),
                        plot.scale = 2,
                        label_genes = all_genes_in_plot,
                        vertice_highlight = (net_pre$meta %>% filter(gtype=='TF') %>% .$gene),
                        edge_hightlight = data.frame(from = c('AHR'),to=c('ADD2')),
                        file_name = 'coef_0.02_subset',
                        vertice_cate_cols = module_colors,
                        pie_custom = pie_dict,
                        output_dir = 'CellOracle/Network_plot/',
                        redo_layout = TRUE,
                        layout_area = 1.1,
                        layout_pulse = 5.45
                       )

NULL
character(0)


Warning message:
“`get.edgelist()` was deprecated in igraph 2.0.0.
ℹ Please use `as_edgelist()` instead.”


pdf 
  2